In [1]:
import requests
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
from datetime import datetime as dt

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [2]:
"""
arg: None
return: driver
"""


def set_driver():
    options = Options()

    # options.set_headless(True)
    options.binary_location = "/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary"
    # options.add_argument("--headless")

    chromedriver_path = "/Users/higashi/Desktop/Document/chromedriver/chromedriver"

    return webdriver.Chrome(options=options, executable_path=chromedriver_path)


In [3]:
"""
arg: url
return: urls
"""


def get_page_links(start_url):
    urls = []
    urls.append(start_url)
    
    url = start_url
    
    for i in range(50):  # 何ページ目まで読むか
        r = requests.get(url)
        html_contents = r.text

        html_soup = BeautifulSoup(html_contents)
        try:
            next_link = 'https://weixin.sogou.com/weixin' + html_soup.find_all('a', {'id': 'sogou_next'})[0].get('href')
            if next_link in urls:
                continue
            else:
                urls.append(next_link)
                url = next_link
        except:
            break

    return urls


In [4]:
"""
arg: driver, url
return: driver, html_soup
"""


def get_page_html_soup_newdriver(driver, url):
    i = 0
    while True:
        if i == 0:
            pass
        else:
            driver = driver
        
        try:
            driver.get(url)
            html = driver.page_source.encode('utf-8')
            html_soup = BeautifulSoup(html, "html.parser")
        
            return driver, html_soup
    
        except TimeoutException:
            driver.quit()
            driver = set_driver()
            timeout = 15
            driver.set_page_load_timeout(timeout)
            # driver = login(driver, url, 'feed')  # ログイン
            i += 1


In [5]:
"""
arg: html_soup
return: feed_id_list
"""


def get_feed_id_list(html_soup):
    feed_id_list = []
    div_elements = html_soup.find_all('ul', {'class': 'news-list'})
    div_elements = div_elements[0].find_all('li')
    for elem in div_elements:
        feed_id = elem.get('d')
        feed_id_list.append(feed_id)
    
    return feed_id_list


In [6]:
"""
arg: html_soup, feed_id
return: feed_link
"""


def get_feed_link(html_soup, feed_id):
    feed_ul = html_soup.find_all('ul', {'class': 'news-list'})
    feed_link = feed_ul[0].find_all('li', {'d': feed_id})[0].find_all('a')[0].get('href')
    
    return feed_link


In [7]:
"""
arg: html_soup, feed_id
return: feed_link
"""


def get_feed_account(html_soup, feed_id):
    feed_ul = html_soup.find_all('ul', {'class': 'news-list'})
    feed_account = feed_ul[0].find_all('li', {'d': feed_id})[0].find_all('a', {'class': 'account'})[0].text
    
    return feed_account


In [8]:
"""
arg:html_soup
return: title
"""


def get_feed_title(html_soup):
    title = ''
    h2_tag = html_soup.find_all('h2', {'class': 'rich_media_title'})
    title_pieces = h2_tag[0].text.split()
    for title_piece in title_pieces:
        title += title_piece
    
    return title


In [9]:
"""
arg: html_soup
return: text
"""


def get_feed_text(html_soup):
    text = ''
    div_tag = html_soup.find_all('div', {'class': 'rich_media_content'})
    p_tags = div_tag[0].find_all('p')
    for p_tag in p_tags:
        text += p_tag.text
    
    return text


In [47]:
"""
STEP1
driver起動
"""

# driver初期化
timeout = 15
driver = set_driver()
driver.set_page_load_timeout(timeout)

# スタートリンクの定義
start_url = 'https://weixin.sogou.com/weixin?type=2&query=d+program&' \
            'ie=utf8&s_from=input&_sug_=n&_sug_type_=&w=01019900&' \
            'sut=1847&sst0=1552552737503&lkt=10%2C1552552735622%2C1552552737399'

driver.get(start_url)


In [48]:
"""
STEP2
データ定義
"""

# 出力データ
feed_id_list = []
feed_links = []
feed_titles = []
feed_texts = []
feed_accounts = []

# 一時データ
feed_id_list_tmp = []

count = 0


In [49]:
"""
STEP3
クローリング開始
記事リンクの取得
"""


while True:
    
    # 現在のurlを取得
    current_url = driver.current_url
        
    # フィードのhtmlを取得
    driver, page_html_soup = get_page_html_soup_newdriver(driver, current_url)
    print('getting page_html_soup was done.')
        
    if page_html_soup:
        feed_id_list_tmp.clear()
        # feed_id_listを取得
        feed_id_list_tmp = get_feed_id_list(page_html_soup)
        print('getting feed_id_list was done.')
        print('{} feed_ids were detected.'.format(len(feed_id_list_tmp)))
        feed_id_list = feed_id_list + feed_id_list_tmp

    for i, feed_id in enumerate(feed_id_list_tmp):
        feed_link = get_feed_link(page_html_soup, feed_id)
        feed_links.append(feed_link)
        feed_account = get_feed_account(page_html_soup, feed_id)
        feed_accounts.append(feed_account)
        
    print('getting_feed_links were done.')
    print('getting_feed_accounts were done.')
    
    count += 1
    print(count)
        
    # nextボタンをクリック
    try:
        driver.find_element_by_id("sogou_next").click()
    except NoSuchElementException as error:
        print(error)
        break


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
1


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
2


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
3


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
4


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
5


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
6


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
7


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
8


getting page_html_soup was done.
getting feed_id_list was done.
10 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
9


getting page_html_soup was done.
getting feed_id_list was done.
1 feed_ids were detected.
getting_feed_links were done.
getting_feed_accounts were done.
10
Message: no such element: Unable to locate element: {"method":"id","selector":"sogou_next"}
  (Session info: chrome=75.0.3732.0)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.14.1 x86_64)



In [51]:
"""
STEP4
クローリング開始
記事テキストの取得
"""

j = 0
for j, feed_link in enumerate(feed_links):
    try:
        driver.get(feed_link)
        driver, feed_html_soup = get_page_html_soup_newdriver(driver, feed_link)
        title = get_feed_title(feed_html_soup)
        text = get_feed_text(feed_html_soup)
        feed_titles.append(title)
        feed_texts.append(text)
        
        print(j)
        j += 1
        print('getting title and text was done.')
    except:
        print('except')
        break


0
getting title and text was done.


1
getting title and text was done.


2
getting title and text was done.


3
getting title and text was done.


4
getting title and text was done.


...

86
getting title and text was done.


87
getting title and text was done.


88
getting title and text was done.


89
getting title and text was done.


90
getting title and text was done.


In [52]:
os.chdir('/Users/higashi/PycharmProjects/Scraping/venv/wechat/data')

df = pd.DataFrame({
    # 'Title': feed_titles,
    # 'Account': feed_accounts,
    # 'ID': feed_id_list,
    'Text': feed_texts
})

df.to_csv('20190314_wechat_dprogram_feed' + '.csv')


In [50]:
print(len(feed_links))


91


908
